In [1]:
# Import dependencies
import requests
import json
import pandas as pd
import csv
from io import StringIO

In [2]:
# Define the base URL for querying the Earthquake layer
query_url = "https://maps.texnet.beg.utexas.edu/arcgis/rest/services/catalog/catalog_all_flat/MapServer/0/query"

# Initialize variables for pagination
start = 0
num = 2000  # Number of records to fetch per request
all_features = []  # List to store all fetched features

while True:
    # Define the parameters for the query, including pagination
    params = {
        "where": "1=1",
        "outFields": "Magnitude,Latitude,Longitude,Event_Date",
        "returnGeometry": True,
        "outSR": "4326",
        "f": "geojson",
        "resultOffset": start,
        "resultRecordCount": num
    }

    # Make the request to the ArcGIS REST API
    response = requests.get(query_url, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        geojson_data = response.json()
        features = geojson_data.get("features", [])
        
        if not features:
            break  # Exit loop if no more features are returned
        
        all_features.extend(features)  # Add the features to the all_features list
        start += len(features)  # Prepare the start value for the next batch
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}, Message: {response.text}")
        break

# Combine all features into a single GeoJSON object
all_geojson_data = {"type": "FeatureCollection", "features": all_features}

# Save the combined GeoJSON data to a file
with open('earthquake_data.geojson', 'w') as f:
    json.dump(all_geojson_data, f, indent=4)

print(f"Earthquake GeoJSON data has been saved to 'earthquake_data.geojson'. {len(all_features)} features were fetched.")


# Extract data from features for DataFrame creation
latitude = [feature['properties']['Latitude'] for feature in all_features]
longitude = [feature['properties']['Longitude'] for feature in all_features]
magnitude = [feature['properties']['Magnitude'] for feature in all_features]
event_date = [feature['properties']['Event_Date'] for feature in all_features]

# Create DataFrame
df_earthquakes = pd.DataFrame({
    'Latitude': latitude,
    'Longitude': longitude,
    'Magnitude': magnitude,
    'Event_Date': event_date
})

# Convert the Event_Date column values into calendar dates and times
df_earthquakes['Event_Date'] = pd.to_datetime(df_earthquakes['Event_Date'], unit='ms')

# Remove times and keep only dates in the 'Event_Date' column
df_earthquakes['Event_Date'] = df_earthquakes['Event_Date'].dt.strftime('%Y-%m-%d')

# Export the DataFrame as a CSV file
df_earthquakes.to_csv('earthquakes.csv', index=False)
print("Earthquake data has been saved to 'earthquakes.csv'.")


Earthquake GeoJSON data has been saved to 'earthquake_data.geojson'. 25096 features were fetched.
Earthquake data has been saved to 'earthquakes.csv'.
